In [1]:
import agentpy as ap
import random
import numpy as np
import pandas as pd
from IPython.display import Image
import networkx as nx
from agentpy.tools import make_list

# For using ggplot from r for better figures 
from pyper import *
r = R(RCMD="/Library/Frameworks/R.framework/Resources/bin/R", use_pandas=True)

Python implementation of the agent-based model from "Sex specific kinship dynamics and the fitness consequences of the social environment in killer whales". The model captures social network dynamics and the evolution of helping and harming behaviour.

In [171]:
class AnimalAgent(ap.Agent):

    def setup(self):
        """ Initialise new agent variables at agent creation """
        self.age = np.random.geometric(1/self.p.N) # *** Why does the rate change with N?
        self.ageClass = self.set_age_class()
        self.sex = random.choice(['m','f'])
        # Random starting values. np array m/f rows (index 0/1), immature/adult (0/1) cols
        self.genome = np.reshape(np.random.choice(['c','d'],4), (2,2))
        self.fitness = 0

    def set_age_class(self):
        """ Set the age class based on a threshold """
        if self.age < self.p.m:
            self.ageClass = 'immature'
        else:
            self.ageClass = 'adult'

    def increase_age(self):
        self.age += 1
        self.set_age_class()

    def get_genome_from_parents(self, parent1_genome, parent2_genome):
        """ Uniform crossover between the two parents to return the child's genome """
        choice = np.random.randint(2, size = parent1_genome.size).reshape(parent1_genome.shape).astype(bool)
        self.genome = np.where(choice, parent1_genome, parent2_genome)

    def mutate_genome(self):
        """ Random uniform mutation of each gene """

        # Create numpy mask to select each gene to mutate with given probability
        probs = [self.p.mut_prob, 1-self.p.mut_prob]
        mask = np.random.choice([True, False], self.genome.shape, p=probs)
        # slip the values for those chosen to mutate
        self.genome[mask] = np.where(self.genome=='d', 'c', 'd')[mask]

    def social_interaction(self):
        """ The agent plays the cooperation game with each social partner """
        # Get the angent's strategy
        # m/f rows (index 0/1), immature/adult cols (0/1)
        row = 0 if self.sex == 'm' else 1
        col = 0 if self.ageClass == 'immature' else 1 
        strategy = self.genome[row,col]
        if strategy == 'c':
            self.fitness -= self.p.cost
            for n in self.network.neighbors(self):
                n.fitness += self.p.benefit
        else:
            return

In [172]:
class MainModel(ap.Model):

    def setup(self):
        """ Initialise the model and setup the agent population """

        # Create agent population
        self.pop = ap.AgentDList(self, self.p.N, AnimalAgent)

        # Create a random undirected network with desired mean degree
        max_edges = (self.p.N * (self.p.N - 1) / 2)
        desired_edges = (self.p.N * self.p.d) / 2
        prob_edge = desired_edges / max_edges
        graph = nx.erdos_renyi_graph(n=self.p.N, p=prob_edge)
        # Define the network in the model and give agents reference to it
        self.network = self.pop.network = ap.Network(self, graph)
        self.network.add_agents(self.pop, self.network.nodes)
    
    def kill_agent(self, agent_to_kill):
        """ Removes the agent and node from the network and deletes agent """

        # Get the node the agent is on and remove the node
        agent_to_kill_node = self.network.positions[agent_to_kill]
        self.network.remove_node(agent_to_kill_node)
        # Remove agent from population
        self.pop.remove(agent_to_kill)

    def birth_agent(self, sex, mother_agent, father_agent):
        """ Creates a single new agent and sets it up in the network
            New agents inherit each gene at random from either parent """

        # Add new agent to the population
        new_agent = ap.AgentDList(self, 1, AnimalAgent)
        new_agent[0].sex = sex
        new_agent[0].get_genome_from_parents(mother_agent.genome, father_agent.genome)
        new_agent[0].mutate_genome()
        self.pop += new_agent
        # Set up the agent on a new node on the network
        self.network.add_agents(new_agent)
        # Give the agent a connection to its mother
        agent_node = self.network.positions[new_agent[0]]
        mother_node = self.network.positions[mother_agent]
        self.network.graph.add_edge(agent_node, mother_node)
        # Give it D random new associates (keeping the mean degree d on ave)
        D = np.random.binomial(self.p.N-2, (self.p.d-1)/(self.p.N-2))
        # Individuals connected to the mother are w times more likely to be selected
        mother_neighbors = list(model.network.neighbors(model.pop[0]))
        mother_neighbor_nodes = [model.network.positions[node] for node in mother_neighbors]
        connected_to_mother = [model.network.positions[agt] in mother_neighbor_nodes for agt in model.pop]
        probs = [self.p.w if connected else 1 for connected in connected_to_mother]
        probs = [ p/sum(probs) for p in probs]
        for i in range(D):
            rand_node = np.random.choice(list(self.network.graph.nodes),p=probs)
            # Check that associate isn't self or already connected
            while (rand_node==agent_node) or (self.network.graph.has_edge(agent_node, rand_node)):
                rand_node = np.random.choice(list(self.network.graph.nodes),p=probs)
            self.network.graph.add_edge(agent_node, rand_node)

    def select_agent_to_kill(self):
        """ Returns which agent to kill.
        Works by selecting agents to survive and seeing who is left
        """
        # Make a list of agent fitnesses aligned with the agents in order
        # Make another list 
        
        return( self.pop.select(self.pop.agent_alive == False) )

    def update(self):
        """ Called after setup and every step """

        # Record stats
        # self.pop.record("opinion")

    def step(self):
        """ Model events per timestep """

        # self.pop.shuffle().opinion_influence()
        self.pop.social_interaction()
        #agent_to_kill = self.select_agent_to_kill()
        #self.kill_agent(agent_to_kill)
        #birth agent **********************
        self.pop.increase_age()

    def end(self):
        """ Termination conditions """
        return

In [173]:
parameters = {
    'N': 20,         # population size ********* What is this?
    'steps': 100,    # Generations
    'm': 10,         # Age of maturity ********************* WE DON'T KNOW WHAT THIS IS
    'w': 28,         # times more likely offspring select mothers' social partner over others
    'd': 19.42,      # Mean degree of the social network
    'mut_prob': 0.1, # Mutation probability
    'benefit': 1,
    'cost': 0.1,
    'fitness_baseline': 0.2
}

In [174]:
# Single run of the model (experiments later)
model = MainModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.059698
Simulation finished


In [41]:
# Get all logged data for agents
results = results.variables.AnimalAgent
# Convert the resulting Pandas series to a dataFrame
results = results.genome.to_frame().reset_index()

AttributeError: variables

In [146]:
model.pop()

TypeError: 'AgentDList' object is not callable

In [158]:

probs = [ p+model.p.baseline_fitness/sum(model.pop.fitness) for p in model.pop.fitness]
np.random.choice(model.pop,19,replace=False,p=probs)

ValueError: probabilities are not non-negative

In [175]:
fitnesses = model.pop.fitness 
fitnesses

[1173.9999999999995, 1173.9999999999995, 1179.5, 1178.3999999999999, 1173.9999999999995, 1173.9999999999998, 1284, 1173.9999999999998, 1173.9999999999998, 1284, 1181.6999999999998, 1173.9999999999998, 1284, 1284, 1173.9999999999998, 1173.9999999999998, 1284, 1284, 1284, 1173.9999999999998]

In [156]:
probs

[10.2, -0.8, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2, 10.2]

In [176]:
model.pop.age

[106, 115, 104, 105, 102, 121, 117, 110, 173, 131, 102, 105, 110, 134, 119, 113, 135, 115, 188, 122]